In [ ]:
#############  u===2   MLP_encode_100 and MLP_encode_1000  :654    ##################
import torch
#import Trainer
from network import NFM
import torch.utils.data as Data
from Utils.criteo_loader import getTestData, getTrainData
from torch.autograd import Variable
from torch.utils.data import DataLoader
from sklearn.metrics import roc_auc_score
from new_nfm_network import NFM
import os
import time
import argparse
import numpy as np
import pandas as pd 
import sys

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.backends.cudnn as cudnn
#3from tensorboardX import SummaryWriter
import sys 
#import network
import config
import evaluate
import data_utils
import Trainer
import torchmetrics

from innvestigator import InnvestigateModel
from utils import Flatten
import torch

import torch.nn as nn
import torch.optim as optim


from torch.utils.data import DataLoader
from torchvision import datasets, transforms

from mnist_test import Net, train, test

input_num=3300
# Network parameters
class Params(object):
    batch_size = 64
    test_batch_size = 20
    epochs = 5
    lr = 0.01
    momentum = 0.5
    no_cuda = True
    seed = 1
    log_interval = 10
    
    def __init__(self):
        pass

args = Params()
torch.manual_seed(args.seed)
#device = torch.device("cpu")
device=torch.device('cuda')
kwargs = {}
nfm_config = \
{
    'n_class':9,
    
    'num_epoch': 500,#训练epoch次数
    'batch_size': 16,#batch_size
    'lr': 1e-3,
    'l2_regularization': 1e-4,
    'device_id': 0,
    'use_cuda': False,
    'epoch':1000,
    
   
    'gene_name':'dataset/qiuguan/origin_800/gene_name.csv',
    'label_name':'dataset/qiuguan/origin_800/gene_label.csv'
    
}

#model definition
import torch.nn as nn
    
    

#model1 = MLP().cuda()
#print(model1)

class MLP_P(nn.Module):
    def __init__(self):
        super().__init__()
        self.bn0=nn.BatchNorm1d(input_num)
        self.fc1 = nn.Linear(input_num, 2000)
        self.bn1= nn.BatchNorm1d(2000)
        self.fc2 = nn.Linear(2000, 100)
        self.bn2=nn.BatchNorm1d(100)
        self.fc3=nn.Linear(100,9)
        self.bn3=nn.BatchNorm1d(9)
        
        self.drop=nn.Dropout(0.3)    


class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.bn0=nn.BatchNorm1d(input_num)
        self.fc1 = nn.Linear(input_num, 2000)
        self.bn1= nn.BatchNorm1d(2000)
        self.fc2 = nn.Linear(2000, 100)
        self.bn2=nn.BatchNorm1d(100)
        self.fc3=nn.Linear(100,9)
        self.bn3=nn.BatchNorm1d(9)
        
        self.drop=nn.Dropout(0.3) 
        
        #self.model1=MLP1().cuda() 
        
        
        
    def forward(self,x):
        
        y1=self.bn0(x)
        y1 = F.relu(self.drop(self.bn1(self.fc1(y1))))
        y1= F.relu(self.drop(self.bn2(self.fc2(y1))))
        return F.softmax(self.bn3(self.fc3(y1)), dim=1)
        
           
        
        
        
   
    
    
    
model = MLP().cuda()

for i in model.state_dict():
    print(i)
print(model)
mlp_paras=list(model.named_parameters())
#print(mlp_paras)
inn_model = InnvestigateModel(model, lrp_exponent=2,
                              method="e-rule",                              beta=.5)

#model1=model
inn_model1 = InnvestigateModel(model, lrp_exponent=2,
                              method="e-rule",
                              beta=.5)



import os
import time
import argparse
import numpy as np
import pandas as pd 

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.backends.cudnn as cudnn
#3from tensorboardX import SummaryWriter
import torch.nn.functional as F  # 激励函数的库
#import network
import config
import evaluate
import data_utils
import Trainer

def one_hot_smoothing(labels, classes, label_smoothing=0.2):
    #n = len(labels)
    n=labels.shape[0]
    eoff = label_smoothing / classes
    output = np.ones((n, classes), dtype=np.float32) * eoff
    for row, label in enumerate(labels):
        output[row, label] = 1 - label_smoothing + eoff
        #print("row:",row,"label:",label)
    return output

def one_hot(labels, classes):
    n = len(labels)
    #eoff = label_smoothing / classes
    output = np.zeros((n, classes), dtype=np.float32)
    for row, label in enumerate(labels):
        output[row, label] = 1
        #print("row:",row,"label:",label)
    return output


import pandas as pd 

from sklearn.model_selection import train_test_split




def one_hot_smoothing(labels, classes, label_smoothing=0.2):
    #n = len(labels)
    n=labels.shape[0]
    eoff = label_smoothing / classes
    output = np.ones((n, classes), dtype=np.float32) * eoff
    for row, label in enumerate(labels):
        output[row, label] = 1 - label_smoothing + eoff
        #print("row:",row,"label:",label)
    return output

def one_hot(labels, classes):
    n = len(labels)
    #eoff = label_smoothing / classes
    output = np.zeros((n, classes), dtype=np.float32)
    for row, label in enumerate(labels):
        output[row, label] = 1
        #print("row:",row,"label:",label)
    return output

class KZDatasetTest(data.Dataset):
    """ Construct the FM pytorch dataset. """
    #def __init__(self, file,label_file, feature_map,n_class=16):
    def __init__(self, csv_path):
    
       
        self.data_info = self.get_data_info(csv_path)
        
        
            
        
        
        

    def __getitem__(self, index):
        # Dataset读取图片的函数
        data, label = self.data_info[index]
        #img = Image.open(img_pth).convert('RGB')
        
        return data, label

    def __len__(self):
        return len(self.data_info)
    
    
    
    
    def get_data_info(self,csv_path):
        #解析路径
        #转为一维list存储，每一位为【图片路径，图片类别】
        labels=[]
        data_info=[]
        df=pd.read_csv(csv_path,sep=',')
        #print("df:",df)
        df=df.iloc[:,1:]
        
        rows,cols=df.shape
        #print(rows,cols)
        for i in df.iloc[:,-1]:
            #print(i)
            labels.append(int(float(i)))
        #print('labels:',labels)
        labels=np.array(labels)
        
        labels=one_hot_smoothing(labels,nfm_config['n_class'])
        #print(labels)
        for i in range(rows):
            data=df.iloc[i,:-1]
            data=data.astype(float)##############
            
            data=np.array(data)##
            
            label=labels[i]
            
            data=torch.from_numpy(data)#
            label=torch.from_numpy(label)#
           
            
            data_info.append((data,label))
        return data_info
    
    

    
import torch
import torch.nn as nn
from torch.utils.data.dataset import *
from PIL import Image
from torch.nn import functional as F
import random
from sklearn.model_selection import train_test_split
import ast
import torchvision


        

class KZDataset(Dataset):
    def __init__(self, csv_path, K,n_class,ki=0, typ='train', transform=None, rand=False):
        
        self.all_data_info = self.get_data_info(csv_path)
        
        if rand:
            random.seed(1)
            random.shuffle(self.all_data_info)
        leng = len(self.all_data_info)
        every_z_len = leng // K
        if typ == 'val':
            self.data_info = self.all_data_info[every_z_len * ki : every_z_len * (ki+1)]
        elif typ == 'train':
            self.data_info = self.all_data_info[: every_z_len * ki] + self.all_data_info[every_z_len * (ki+1) :]
            
        self.transform = transform
        
        

    def __getitem__(self, index):
        # Dataset读取图片的函数
        data, label = self.data_info[index]
        #img = Image.open(img_pth).convert('RGB')
        
        return data, label

    def __len__(self):
        return len(self.data_info)
       
    
    def get_data_info(self,csv_path):
        #解析路径
        #转为一维list存储，每一位为【图片路径，图片类别】
        labels=[]
        data_info=[]
        df=pd.read_csv(csv_path,sep=',')
        #print("df:",df)
        df=df.iloc[:,1:]
        #print("df:",df)
        print(df.shape)
        #print("df:",df)
        #print(df.iloc[:,-1])
        #df=df.applymap(ast.literal_eval)
        rows,cols=df.shape
        #print(rows,cols)
        for i in df.iloc[:,-1]:
            #print(i)
            labels.append(int(float(i)))
        #print('labels:',i,labels)
        labels=np.array(labels)
        #print('labels:',labels)
        #labels=np.array(labels)
        labels=one_hot_smoothing(labels,nfm_config['n_class'])
        #print(labels)
        for i in range(rows):
            data=df.iloc[i,:-1]
            data=data.astype(float)##############
            #print("i,data:",i,data)
            #data=pd.DataFrame(data,dtype=float)###############
            data=np.array(data)##
            
            label=labels[i]
            #print(data.shape)
            #print(label.shape)
            #label=label.tolist()
            data=torch.from_numpy(data)#
            label=torch.from_numpy(label)#
           
            
            data_info.append((data,label))
        return data_info

    

        
from torch.autograd import Variable
from torch.utils.data import DataLoader
from sklearn.metrics import roc_auc_score
from new_nfm_network import NFM
import os
import time
import argparse
import numpy as np
import pandas as pd 
import sys

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.backends.cudnn as cudnn
#3from tensorboardX import SummaryWriter
import sys 
#import network
import config
import evaluate
import data_utils
import Trainer
import torchmetrics
#LRP
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
%matplotlib inline

from innvestigator import InnvestigateModel
from utils import Flatten
def standN(x):
    row,col=x.shape
    max_x=torch.max(x,1)
    min_x=torch.min(x,1)
    #print(max_x)
    #print(min_x)
    for i in range(row):
        x[i]=(x[i]-min_x.values[i])/max_x.values[i]
    return x

def standNorm(x):
    row,col=x.shape
    mean=torch.mean(x,1)
    std=torch.std(x,1)
    #print(mean)
    #print(std)
    for i in range(row):
        x[i]=(x[i]-mean[i])/std[i]
    return x

def train_epoch(kii,epoch,train_loader,batch_size,optimizer,loss_func,loss2_list):
    BATCH_SIZE=batch_size
    total = 0
    correct=0
    total_loss=0
    #loss_score=torch.tensor([[]]).cuda()
    #
    #loss_op=0
    #oss2_list=[]
    model.train()
    total_train_accuracy=0  
    BL=nn.BatchNorm1d(input_num)
    BL=BL.cuda()
    for batch_idx, (x, labels) in enumerate(train_loader):
            
        labels = Variable(labels)
        x = Variable(x)
            
        x_row,x_col=x.shape   
        x=torch.tensor(x,dtype=torch.float)
        #print(x.shape)
        labels=torch.tensor(labels,dtype=torch.float)
        x, labels = x.cuda(), labels.cuda()
        labels_int=labels=torch.max(labels,1)[1]#########
        #print(labels_int)
        #print('labels_int:',labels_int.shape)
        #print('labels:',labels) 
        #print('x:',x.shape)
        #xm=x.clone()
        xm=x.clone().detach().cuda()
        xn=x.clone().detach().cuda()
        #loss2=0
        if kii>=0: 
            
            #inn_model2.evaluate(in_tensor=xn)
            #inn_model1.evaluate(in_tensor=xm)
            
            model_prediction, only_max_score,org_shape = inn_model.innvestigatex(in_tensor=x)
            model_prediction.cuda()
            
            #model_prediction1, only_max_score1,org_shape1= inn_model1.innvestigatex()
            #model_prediction1.cuda()
            
            #only_max_score1=only_max_score.clone().detach()
            
            
            #print('only_max_score:',only_max_score)
            #only_max_score1=only_max_score.detach().clone()
            #print(labels.shape,labels)   
        
            #print("torch.argmax(labels[batch_idx]):",torch.argmax(labels[batch_idx]))
            #print(model_prediction.shape, model_prediction)
            #model_prediction.cuda()
            #print('input_relevance_values:')
            rel_for_class_list=labels
            rel_for_class_list1=labels
            para_list=[]
            para_orign_list=[0]*batch_size
            #print('inn_model1')
            #input_relevance_values1,layers1=inn_model2.compute_relevance_scorex(only_max_score1,rel_for_class_list1,org_shape,para_orign_list)
            ##input_relevance_values1,layers1=inn_model2.compute_relevance_scorex(only_max_score1,rel_for_class_list1,org_shape,para_orign_list)
            
            
            for i in range(batch_size):
            
                max_pre=torch.argmax(model_prediction[i])
                if max_pre!=labels[i]:
                    #print('only_max_score1:',i,only_max_score)
                    #only_max_score=only_max_score1
                    #input_relevance_values=inn_model.compute_relevance_score(only_max_score,labels[i],org_shape,para=0.1)
                    #rel_for_class=labels[i]
                    para=random.gauss(c1,c2)#(kii>0,93.75)(ki>1,92.1875)
                    #para=random.gauss(0.6,0.2)#96.8  92.2
                    #para=0.6#(kii>0:93.75;>=0:89.0625)
                    #para=0.4(kii>0,93.75)
                    
                    #para=random.uniform(0.3,0.8)#(kii=0,89.0625)
                    para=round(para,4)
                    #para=0.2
                    para_list.append(para)
                    #print('device:',input_relevance_values.device)
                    #input_relevance_values[labels[i]]=input_relevance_values[labels[i]].cuda()
                    #input_relevance_values[labels[i]]=input_relevance_values[labels[i]].exp()
                    #print('input_relevance_values:',i, input_relevance_values)
                    #print('input_relevance_values[labels[i]]:',i,batch_idx,input_relevance_values[labels[i]])
                    #cc[i]=torch.mul(x[i],input_relevance_values[labels[i]])+torch.tensor(0.1,dtype=torch.float)##############注意力
                    #print('pre_target:',pre_target[i])
                    #print('cc',i,cc[i].shape)
                else:
                    #print('only_max_score2:',i,only_max_score)
                    #only_max_score=only_max_score1
                    #input_relevance_values=inn_model.compute_relevance_score(only_max_score,labels[i],org_shape,para=0)
                    #input_relevance_values[labels[i]]=input_relevance_values[labels[i]].exp()
                    #print('input_relevance_values:',i, input_relevance_values)
                    #print('input_relevance_values[i]:',i,batch_idx,input_relevance_values[labels[i]])
                    #cc[i]=torch.mul(x[i],input_relevance_values[labels[i]])##############注意力
                    #print('cc',i,cc[i].shape)
                    para=0
                    para=round(para,4)
                    para_list.append(para)
            #print(para_list)
            #print(para_orign_list)
            input_relevance_values,layers=inn_model.compute_relevance_scorex(only_max_score,rel_for_class_list,org_shape,para_list)
            #print('inn_model2')
            #input_relevance_values,input_relevance_values1,layers,layers1=inn_model2.compute_relevance_scorey(only_max_score,only_max_score1,rel_for_class_list,org_shape,para_list,para_orign_list)
            #input_relevance_values=input_relevance_values.exp()############
            #input_relevance_values1=input_relevance_values1.exp()############
            
            
            #input_relevance_values1=input_relevance_values1[-1]
            input_relevance_values=input_relevance_values[-1]
            #sum_input_relevance_values=torch.sum(input_relevance_values)
            #sum_input_relevance_values1=torch.sum(input_relevance_values1)
            
            
            #input_relevance_values=F.softmax(input_relevance_values/sum_input_relevance_values)
            #input_relevance_values1=F.softmax(input_relevance_values1/sum_input_relevance_values1)
            
            input_relevance_values=input_relevance_values.exp()############
            #input_relevance_values1=input_relevance_values1.exp()
            
            #loss2=torch.abs(input_relevance_values1-input_relevance_values)
            #loss2=torch.sum(loss2)
            
            
            #loss2_list.append(loss2)
            
            ee=[[0]*input_num]*batch_size
            ee=torch.tensor(ee,dtype=torch.float)
        
            ee=ee.cuda()
            
            for i in range(batch_size):
                        max_pre=torch.argmax(model_prediction[i])######
                        if max_pre!=labels[i]: #and i==min_predict_mis:###
                            
                                
            
                                ee[i]=torch.mul(x[i],input_relevance_values[-1][i])##############注意力33333333333333333weight
                                #dd=torch.mul(bias,input_relevance_values[-1][i])############bias
                                #print('cc:',i,cc)
                                #ee[i]=torch.mul(x[i],input_relevance_values[-1][i])
                                ################
            
            
            
            
            
            ee=torch.mul(x,input_relevance_values[-1])
            
            x=torch.add(x,ee)
            '''
            ee=torch.mul(x,input_relevance_values)
            #print('ee:',ee)
            uu=F.softmax(ee/torch.sum(ee))
            #print('uu:',uu)
            x=torch.add(x,uu)
            
            '''
           
            #print('x:',x)
            
            '''
            input_len=len(input_relevance_values)
            #input_relevance_values1,layers1=inn_model.compute_relevance_scorex(only_max_score,rel_for_class_list,org_shape,para_orign_list)
            ee=[[0]*input_num]*batch_size
            ee=torch.tensor(ee,dtype=torch.float)
        
            ee=ee.cuda()
            #sum_input_relevance_values=torch.sum(input_relevance_values,dim=0)
            #input_relevance_values=F.softmax(input_relevance_values/sum_input_relevance_values,dim=1)
            input_relevance_values[-1]=F.softmax(input_relevance_values[-1],dim=1)################
        
            
            for i in range(batch_size):
                        max_pre=torch.argmax(model_prediction[i])######
                        if max_pre!=labels[i]: #and i==min_predict_mis:###
                            
                                
            
                                ee[i]=torch.mul(x[i],input_relevance_values[-1][i])##############注意力33333333333333333weight
                                #dd=torch.mul(bias,input_relevance_values[-1][i])############bias
                                #print('cc:',i,cc)
                                #ee[i]=torch.mul(x[i],input_relevance_values[-1][i])
                                ################
            
            
            
            
            
            #ee=torch.mul(x,input_relevance_values[-1])
            
            if epoch%3==0:
                x=torch.subtract(x,ee)
            else:
                
                x=torch.add(x,ee)
        
            
            
            '''
            
            
            
            #x=torch.add(x,ee)
            
            
                    
            
            
            
            
              
            
            
                
                
                
                        
        
        
        
       
        
        
        
         
        #print('x_new:',x)
        #print('new_x:',x)
        optimizer.zero_grad()
        y_predict=model(x)
        #y_predict=model(x,labels)
            
        
        
        loss1 = loss_func(y_predict, labels)
        #print('loss1:',loss1)
        #loss2=loss_func(y_predict1,labels)
        #loss2=u*(1/loss_op)
        #print('input_relevance_values:',input_relevance_values.shape)
        #print('loss_score:',loss_score.shape)
        #cc=1.0/torch.abs(torch.sum(cc))
        
        #print('loss1:',loss1)
        #print('loss2:',loss2)
        
        
        #loss2=u*cc
        #print('input_relevance_values:',input_relevance_values)
        
        #print('loss1:',loss1)
        #print('loss2:',loss2)
        #loss=loss1+loss2
        
        #u=10000
        #loss=loss1+u*loss2
        loss=loss1
        #loss = loss_func(y_predict, labels)
        loss.backward()
        optimizer.step()
        #loss2_list.append(u*loss2)   
        loss = loss.item()
           

        total_loss += loss
            
            
            
        batch_train_acc=torchmetrics.functional.accuracy(y_predict,labels_int)
        total_train_accuracy+=batch_train_acc
    #plotLoss(loss2_list,batch_idx+1)   #################################     
    total_train_accuracy/=(batch_idx+1)
    print('total_train_accuracy:',total_train_accuracy)
    print("Training Epoch: %d, total loss: %f" % (epoch_id, total_loss))
    return total_loss,total_train_accuracy,loss2_list

def val_epoch(test_loader,batch_size,optimizer): 
    batch_size_num=0
    total_test_acc=0
    model.eval()
    for i , (inputs , targets) in enumerate(test_loader):   
        print("val")
            
            
        inputs = Variable(inputs)   
        targets = Variable(targets)     
           
        inputs=torch.tensor(inputs ,dtype=torch.float)   
        targets=torch.tensor(targets ,dtype=torch.float)   
        inputs , targets = inputs.cuda(),  targets.cuda()
        targets=torch.max(targets,1)[1]
        
        
        
        
        
        optimizer.zero_grad()
        #yhat = model1(inputs,targets)
        yhat=model(inputs)
            
            
            
        #targets=torch.max(targets,1)[1]
            
            
            
        batch_test_acc=torchmetrics.functional.accuracy(yhat,targets)
            
        total_test_acc+=batch_test_acc
            
        batch_size_num=i
    total_test_acc/=(batch_size_num+1)
        ###print('total_test_accuracy:',total_test_acc/(batch_size+1))
    print('total_test_accuracy:',total_test_acc)
        
                    
                    
            
            
    
        
   
    
    return total_test_acc


import numpy as np
import math
import matplotlib.pyplot as plt

def plotLoss(loss,epoch):
    plt.rcParams['font.sans-serif']=['SimHei']
    plt.rcParams['axes.unicode_minus'] = False
    plt.figure(figsize=(8, 5))
    x=[i for i in range(epoch)]
    #acc_train=acc_train.cpu()
    #acc_test=acc_test.cpu()
    plt.plot(x, loss, 'r-', mec='k', label='Logistic Loss', lw=2)
    #plt.plot(x,acc_train,'b-',mec='k',label='accuracy Train',lw=2)
    #plt.plot(x,acc_test,'g-',mec='k',label='accuracy Test',lw=2)
    #plt.plot(x, y_01, 'g-', mec='k', label='0/1 Loss', lw=2)
    #plt.plot(x, y_hinge, 'b-',mec='k', label='Hinge Loss', lw=2)
    #plt.plot(x, boost, 'm--',mec='k', label='Adaboost Loss',lw=2)
    plt.grid(True, ls='--')
    plt.legend(loc='upper right')
    plt.title('损失函数')
    plt.show()

In [ ]:
#_,model=MLPA().cuda
'''
inn_model = InnvestigateModel(model1, lrp_exponent=2,
                              method="e-rule",
                              beta=.5)

'''
import itertools
num=0
K=10
test_metrics=[]
train_loss_total_list=[]
loss2_list=[]
for ki in range(K):
    trainset = KZDataset(csv_path='dataset/qiuguan/origin_800/xiaoqiu_xiaoguan/train_val_info.csv',K=K, n_class=nfm_config['n_class'],ki=ki,  typ='train', transform=None, rand=True)
    valset = KZDataset(csv_path='dataset/qiuguan/origin_800/xiaoqiu_xiaoguan/train_val_info.csv', K=K,n_class=nfm_config['n_class'],ki=ki,  typ='val', transform=None, rand=True)
    train_loader = data.DataLoader(
         dataset=trainset,
         #transform=torchvision.transforms.ToTensor(),
         drop_last=True,
         batch_size=nfm_config['batch_size'],
         shuffle=True)
    val_loader = data.DataLoader(
         dataset=valset,
         #transform=torchvision.transforms.ToTensor(),
         drop_last=True,
         batch_size=nfm_config['batch_size']
        
     )
    
    model_path='dataset/qiuguan/origin_800/non_encode_attention/new/para0.4_0.2/'
    #BATCH_SIZE=batch_size
    optimizer = torch.optim.Adam(model.parameters(), lr=nfm_config['lr'], weight_decay=nfm_config['l2_regularization'])
    #total = 0
    
    
    loss_func=torch.nn.CrossEntropyLoss()
    
    
   
    model_named_parameters=[j for j in model.named_parameters()]
    #print('model_parameters:',model_named_parameters)
    #print('model.state.dict:',model.state_dict())
    epoches=101
    for epoch_id in range(epoches):
          
        
        
        train_loss_total,acc_train,loss2_list=train_epoch(ki,epoch_id,train_loader,nfm_config['batch_size'],optimizer,loss_func,loss2_list)
        train_loss_total_list.append(train_loss_total)#
        if epoch_id %10==0:
            num=num+1
            path=os.path.join(model_path,'MLP'+str(num)+str(K)+'.pkl')
            torch.save(model.state_dict(),path)
    print("the ",ki," epoch ends")
    plotLoss(train_loss_total_list,epoches)
    
    train_loss_total_list=[]
    acc_test=val_epoch(val_loader,nfm_config['batch_size'],optimizer)
    print("acc_test_each_k:",acc_test)
    test_metrics.append(acc_test)

print(test_metrics)
#test_metrics=test_metrics.tolist()
test_metrics=[x.cpu().detach().numpy() for x in test_metrics]
print(test_metrics)
acc_test_metrics=np.mean(test_metrics) 
print("acc_test_metrics:",acc_test_metrics)

#print(list(mlp.named_parameters()))

path=os.path.join(model_path,'MLP'+str(num)+str(12345)+'.pkl')
torch.save(model.state_dict(),path)

In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt

def plotAcc(len_list,acc_score_list):
    plt.rcParams['font.sans-serif']=['SimHei']
    plt.rcParams['axes.unicode_minus'] = False
    plt.figure(figsize=(8, 5))
    #x=[i for i in range(lenth)]
    #acc_train=acc_train.cpu()
    #acc_test=acc_test.cpu()
    plt.plot(len_list, acc_score_list, 'r-', mec='k', label='Logistic Loss', lw=2)
    #plt.plot(x,acc_train,'b-',mec='k',label='accuracy Train',lw=2)
    #plt.plot(x,acc_test,'g-',mec='k',label='accuracy Test',lw=2)
    #plt.plot(x, y_01, 'g-', mec='k', label='0/1 Loss', lw=2)
    #plt.plot(x, y_hinge, 'b-',mec='k', label='Hinge Loss', lw=2)
    #plt.plot(x, boost, 'm--',mec='k', label='Adaboost Loss',lw=2)
    plt.grid(True, ls='--')
    plt.legend(loc='upper right')
    plt.title('损失函数')
    plt.show()
    
testset_guan = KZDatasetTest(csv_path='dataset/qiuguan/origin_800/xiaoguan/test_info.csv')
   
test_loader_guan = data.DataLoader(
         dataset=testset_guan,
         #transform=torchvision.transforms.ToTensor(),
         drop_last=True,
         batch_size=nfm_config['batch_size']
        
     )

testset_qiu = KZDatasetTest(csv_path='dataset/qiuguan/origin_800/xiaoqiu/test_info.csv')
   
test_loader_qiu = data.DataLoader(
         dataset=testset_qiu,
         #transform=torchvision.transforms.ToTensor(),
         drop_last=True,
         batch_size=nfm_config['batch_size']
        
     )
input_num=3300

In [ ]:
##################149特征基因运行后的示意图
################################################特征基因个数为300

##############小球测试


import torch

#功能：加载保存到path中的各层参数到神经网络

#path='dataset/qiuguan/origin_800/LRP/non_encode/40/attention0/MLP1112345.pkl'

#nfm=NFM(nfm_config)
mlp=MLP()
#print(nfm)
#net = nn.DataParallel(net)
#net = net.to(device)
mlp.load_state_dict(torch.load(path),strict=False)
mlp.cuda()

print(mlp)


#print(model.state_dict())

mlp_params = list(mlp.named_parameters())
#print(mlp_params)
net=mlp


#net=model

testset_guan = KZDatasetTest(csv_path='dataset/qiuguan/origin_800/xiaoguan/test_info.csv')
   
test_loader_guan = data.DataLoader(
         dataset=testset_guan,
         #transform=torchvision.transforms.ToTensor(),
         drop_last=True,
         batch_size=nfm_config['batch_size']
        
     )

testset_qiu = KZDatasetTest(csv_path='dataset/qiuguan/origin_800/xiaoqiu/test_info.csv')
   
test_loader_qiu = data.DataLoader(
         dataset=testset_qiu,
         #transform=torchvision.transforms.ToTensor(),
         drop_last=True,
         batch_size=nfm_config['batch_size']
        
     )

In [ ]:
from PlotCurves import plotGraph,plotLoss,plotMatrix
from train_val_test import evaluate_model
import os
path_dir='dataset/qiuguan/origin_800/non_encode_attention/new/para0.4_0.2/'
mlp=MLP()
path_list=[]
acc_score_list=[]
from sklearn.metrics import precision_recall_curve, average_precision_score,roc_curve, auc, precision_score, recall_score, f1_score, confusion_matrix, accuracy_score 
pos=0
for parent, _, files in os.walk(path_dir):
    pathes=files
for path_ in pathes:
    if path_.startswith('MLP'):  # 去除隐藏文件


        
        path=os.path.join(path_dir,path_)
        print(path)
        mlp.load_state_dict(torch.load(path),strict=False)
        mlp.cuda()

        net=mlp
        actuals,predictions,acc_test=evaluate_model(test_loader_guan,net,input_num)

       
        target_list=actuals 
        pred_list=predictions 
      
        y_true=target_list 
        y_pred=pred_list 
      

        acc_score=accuracy_score(y_true=target_list, y_pred=pred_list) 

        acc_score_list.append(acc_score)
        path_list.append(path)
        pos=pos+1
max_guan=max(acc_score_list)
print(max_guan)
#max_len=len(max_qiu)
max_guan_list=[]
max_guan_len_list=[]
for i,score in enumerate(acc_score_list):
    
    if score==max_guan:
        print(i,path_list[i],score)
        
        max_guan_list.append(score)
        max_guan_len_list.append(i)
#lenth=len(acc_score_list)

plotAcc(max_guan_len_list,max_guan_list)